In [1]:
from __future__ import print_function
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # warning level
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # use GPU 0
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from utilities import *
from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=sys.maxsize)

In [10]:
def runModel(ID):
    global sess
    global predY
    global testAcc
    correct=0
    wrong=0

    # Set configs
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:

        # Restore model weights from previously saved model
        model_path_ID = model_path + ID + '/'
        saver = tf.train.import_meta_graph(model_path_ID+'model.ckpt.meta')
        saver.restore(sess,tf.train.latest_checkpoint(model_path_ID))
        print("Model of %s is restored" % str(ID+'A_'+str(run)))

        # Restore model variables from previously saved model
        graph = tf.get_default_graph()
        accuracy = graph.get_tensor_by_name("Accuracy:0")
        x1 = graph.get_tensor_by_name("x1:0")
        x2 = graph.get_tensor_by_name("x2:0")
        y = graph.get_tensor_by_name("y:0")
        pred = graph.get_tensor_by_name("pred:0")

        # run the model with test data
        testAcc, predY = sess.run([accuracy, pred], feed_dict={x1: testX1, y: testYcat, x2: testX2})
        predClass = np.argmax(predY,1)
        actClass  = np.argmax(testYcat,1)
        for i in range(0,predClass.shape[0]):
            if predClass[i]==actClass[i]:
                correct=correct+1
            else:
                wrong=wrong+1
        print("correct=",correct,"wrong=",wrong)
            
        #print("predY=",predClass)
        #print("testY=",actClass)
            
        # save the predictions
        directory = os.path.dirname(predsPath)
        try:
            os.stat(directory)
        except:
            os.mkdir(directory)
        np.save( predsPath + ID + '_predY.npy', predY)

    return



In [3]:
def personalResults(): # this function calculates the confusion matrix for each ID
    predClass = np.argmax(predY,1)
    C = confusion_matrix(testY.ravel(), predClass.ravel(), np.arange(n_classes))
    np.set_printoptions(suppress=True)
    return C


In [4]:
def loadTestVars(ID):
    global testX1
    global testX2
    global testY
    global testYcat

    # load test variables
    vars_path_ID = vars_path + ID + '/'
    testX = np.load(vars_path_ID+'testX.npy')
    testY = np.load(vars_path_ID+'testY.npy')
    testYcat = np.load(vars_path_ID+'testYcat.npy')

    # mask some parts to make the desired signal
    mask1 = np.zeros((780,), dtype=bool)
    mask2 = np.zeros((780,), dtype=bool)
    mask1[0:4]=1
    mask1[0+390:4+390]=1
    mask1[4:4+250+1]=1
    mask1[4+390:4+390+250+1]=1
    mask2[0:4]=1
    mask2[0+390:4+390]=1
    mask2[4+250+1:4+250+1+135]=1
    mask2[4+250]=1
    mask2[4+390+250+1:4+390+250+1+135]=1
    mask2[4+390+250]=1
    testX1 = testX[:,:,mask1]
    testX2 = testX[:,:,mask2]

    # make some general numbers
    num_examples = testX.shape[0]
    n_input1 = (testX1.shape[2]//n_steps1)
    n_input2 = (testX2.shape[2]//n_steps2)
    testX1 = testX1.reshape((num_examples, n_steps1, n_input1))
    testX2 = testX2.reshape((num_examples, n_steps2, n_input2))

    return

In [5]:
n_classes = 7
n_steps = 10
n_hidden = 30
ret=0
n_steps1 = n_steps
n_steps2 = n_steps

# data paths
vars_path = '../Data/AA/'
models_path = '../models/'
res_path = models_path + 'resA.txt'
os.system('rm ' + res_path)
IDs = ['100','101','103','105','106','108','109','200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215'] # all records
#IDs = ['100','101','103','105','106','108','109','111','112','113','114','115','116','117','118','119','121','122','123','124','200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234'] # all records
#IDs = ['200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']
runs = np.random.permutation(np.arange(int(1), int(10)))#ret#['_1', '_2', '_3', ..., '_50']
outArr = np.zeros((1,8)) # array of 8 elements for the output


Here we will load model-alpha from memory and pass X into it and we will get prediction. Prediction will be a 1-D vector (for each input) having length equal to number of classes. We will save this prediction as it is to blend alpha and beta output.
To get the prediction result of alpha we can check for max value in prediction which will correspond to predicted class.

In [11]:
for run in runs:

    # path of saving models and their results
    model_path = models_path + 'modelsA_' + str(run) + '_' + str(ret) + '/'
    predsPath = '../preds/testA_outs_' + str(run) + '_' + str(ret) + '/'
    allCs = {}

    # loop over all patients
    for ID in IDs:
        loadTestVars(ID) #load variables	
        tf.reset_default_graph() #reset the graph
        runModel(ID) #run the full model with new test data
        C = personalResults() #see the results
        allCs[ID] = C #store confusion matrices

    #outArr += calc_tables(allCs, n_classes)

#outArr /= np.size(runs)


INFO:tensorflow:Restoring parameters from ../models/modelsA_7_0/100/model.ckpt
Model of 100A_7 is restored
correct= 1902 wrong= 0
INFO:tensorflow:Restoring parameters from ../models/modelsA_7_0/101/model.ckpt
Model of 101A_7 is restored
correct= 1518 wrong= 5
INFO:tensorflow:Restoring parameters from ../models/modelsA_7_0/103/model.ckpt
Model of 103A_7 is restored
correct= 1725 wrong= 4
INFO:tensorflow:Restoring parameters from ../models/modelsA_7_0/105/model.ckpt
Model of 105A_7 is restored
correct= 2144 wrong= 11
INFO:tensorflow:Restoring parameters from ../models/modelsA_7_0/106/model.ckpt
Model of 106A_7 is restored
correct= 1693 wrong= 3
INFO:tensorflow:Restoring parameters from ../models/modelsA_7_0/108/model.ckpt
Model of 108A_7 is restored
correct= 1473 wrong= 7
INFO:tensorflow:Restoring parameters from ../models/modelsA_7_0/109/model.ckpt
Model of 109A_7 is restored
correct= 2088 wrong= 11
INFO:tensorflow:Restoring parameters from ../models/modelsA_7_0/200/model.ckpt
Model of 

correct= 1724 wrong= 5
INFO:tensorflow:Restoring parameters from ../models/modelsA_2_0/105/model.ckpt
Model of 105A_2 is restored
correct= 2145 wrong= 10
INFO:tensorflow:Restoring parameters from ../models/modelsA_2_0/106/model.ckpt
Model of 106A_2 is restored
correct= 1696 wrong= 0
INFO:tensorflow:Restoring parameters from ../models/modelsA_2_0/108/model.ckpt
Model of 108A_2 is restored
correct= 1472 wrong= 8
INFO:tensorflow:Restoring parameters from ../models/modelsA_2_0/109/model.ckpt
Model of 109A_2 is restored
correct= 2085 wrong= 14
INFO:tensorflow:Restoring parameters from ../models/modelsA_2_0/200/model.ckpt
Model of 200A_2 is restored
correct= 2132 wrong= 36
INFO:tensorflow:Restoring parameters from ../models/modelsA_2_0/201/model.ckpt
Model of 201A_2 is restored
correct= 1214 wrong= 307
INFO:tensorflow:Restoring parameters from ../models/modelsA_2_0/202/model.ckpt
Model of 202A_2 is restored
correct= 1809 wrong= 62
INFO:tensorflow:Restoring parameters from ../models/modelsA_2

Model of 108A_9 is restored
correct= 1472 wrong= 8
INFO:tensorflow:Restoring parameters from ../models/modelsA_9_0/109/model.ckpt
Model of 109A_9 is restored
correct= 2086 wrong= 13
INFO:tensorflow:Restoring parameters from ../models/modelsA_9_0/200/model.ckpt
Model of 200A_9 is restored
correct= 2132 wrong= 36
INFO:tensorflow:Restoring parameters from ../models/modelsA_9_0/201/model.ckpt
Model of 201A_9 is restored
correct= 1199 wrong= 322
INFO:tensorflow:Restoring parameters from ../models/modelsA_9_0/202/model.ckpt
Model of 202A_9 is restored
correct= 1810 wrong= 61
INFO:tensorflow:Restoring parameters from ../models/modelsA_9_0/203/model.ckpt
Model of 203A_9 is restored
correct= 2391 wrong= 90
INFO:tensorflow:Restoring parameters from ../models/modelsA_9_0/205/model.ckpt
Model of 205A_9 is restored
correct= 2168 wrong= 33
INFO:tensorflow:Restoring parameters from ../models/modelsA_9_0/207/model.ckpt
Model of 207A_9 is restored
correct= 1384 wrong= 208
INFO:tensorflow:Restoring para